# Mock Data Creation

Mock data are required as is for the MeerKLASS cross-correlation power spectrum analysis (see e.g. [Cunnington+2023b](https://ui.adsabs.harvard.edu/abs/2023MNRAS.523.2453C/abstract) and/or [MeerKLASS Collaboration 2024](https://arxiv.org/abs/2407.21626)).  For now, the MeerKLASS data associated with these publications are still private.  We (UKSRC) were granted permission to distribute and analyze the mock data in place of the real data in this pipeline to enable sharing of the pipeline with the wider SRCNet.  Sky coordinates (RA, Dec) and pixel counts are important metadata for the pipeline, and these metadata are stored in the FITS files of the observed MeerKLASS data.  This notebook documents the process taken to replace the private data in these FITS files with mock data.  In this way, the generated FITS files contain the appropriate metadata but no traces of the real data.

**NOTE:** The only step not documented in this notebook was creating copies of the MeerKLASS FITS files.  The original data files were copied to the `ska-teal-21cm-map` tenancy on Azimuth where Jacob Burba was given permission by MeerKLASS to copy and analyze the private MeerKLASS data.  This step has thus been excluded from this notebook to maintain the privacy of the original data.

# Imports and function definitions

In [1]:
%matplotlib inline

import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt

from astropy import units
from astropy.io import fits

In [2]:
# This function was copied from the meerklass/meerpower github repo
# https://github.com/meerklass/meerpower
# from the meerpower/Init.py module

def cal_freq(ch):
    """
    Converts a L-band channel index to a frequency in Hertz.

    Parameters
    ----------
    ch : int
        L-band frequency channel index.

    Returns
    -------
    freq : float
        Freuqency in Hertz.

    """
    # Function from Jingying Wang to get L-band channel frequencies
    v_min = 856.0
    v_max = 1712.0
    dv = 0.208984375
    assert((v_max - v_min)/dv == 4096)
    freq_MHz = ch*dv + v_min
    freq = freq_MHz*1e6

    return freq

# Frequency axis definition

In [3]:
# Extrema for frequencies used in the MeerKLASS
# cross-correlation power spectrum pipeline in MHz
freq_min = 971 * units.MHz
freq_max = 1023.8 * units.MHz

# Frequency channel indices (4096 frequency channels)
freq_chans = np.arange(1, 4096+1, 1)
# Frequencies in MHz
freqs = units.Quantity(cal_freq(freq_chans), unit='Hz').to('MHz')
freq_mask = np.logical_and(freqs > freq_min, freqs < freq_max)

# Replace counts with uniform counts

In [9]:
true_counts_path = Path('../lband/2021/Nscan961_Npix_count_cube_p0.3d_sigma4.0_iter2.fits')
with fits.open(true_counts_path) as hdul:
    # The counts array has shape (Npix_ra, Npix_dec, Nfreqs)
    counts = hdul[0].data
    print(counts.shape)

# Filepath where the mock data counts will be saved
# This file is a copy of the fits file in `true_counts_path` to preserve the metadata
mock_counts_path = Path('data/Npix_counts.fits')

(133, 73, 4096)


In [5]:
# Load the mock data to construct the pixel counts footprint as the
# data and the mock data have slightly different footprints in RA, Dec
mock_data_path = Path('../lband/2021/dT_HI_p0.3d_wBeam_499.npy')
mock_data = np.load(mock_data_path)

In [ ]:
mock_counts = np.zeros_like(counts)

# Construct mask which is only nonzero for
# the pixels where the mock data is nonzero
nonzero_data_mask = mock_data.std(axis=-1) > 0
mock_counts[..., freq_mask] = nonzero_data_mask.astype(float)[:, :, None]

with fits.open(mock_counts_path, mode='update') as hdul:
    hdul[0].data = mock_counts

# Replace data with mock data

In [103]:
true_data_path = Path('../lband/2021/Nscan961_Tsky_cube_p0.3d_sigma4.0_iter2.fits')
mock_data_path_npy = Path('../lband/2021/dT_HI_p0.3d_wBeam_499.npy')
with fits.open(true_data_path) as hdul:
    # The data array has shape (Npix_ra, Npix_dec, Nfreqs)
    data = hdul[0].data
    print(data.shape)
mock_data = np.load(mock_data_path_npy)
# mock_data[mock_data == 0] = np.nan
print(mock_data.shape)

# Filepath for mock data fits file
mock_data_path = Path('data/Tsky.fits')

(133, 73, 4096)
(133, 73, 252)


In [104]:
# Load mock counts to get pixel indices for non-NaN mock data
with fits.open('data/Npix_counts.fits') as hdul:
    mock_counts = hdul[0].data
print(mock_counts.shape)

(133, 73, 4096)


In [ ]:
mock_data_out = np.zeros_like(data)

# Only place mock data in the pixels where the mock counts are nonzero
mock_data_mask = mock_counts > 0
mock_data_out[mock_data_mask] = mock_data[mock_data_mask[..., freq_mask]]

with fits.open(mock_data_path, mode='update') as hdul:
    hdul[0].data = mock_data_out